*4*/13/2023:
Experiment
adding recall, precision, accuracy, f1 score, tn, tp, fn, fp





4/1/2023: Change Label to Subject and Patient
FP1 + CZ signals

In [38]:
pip install mne

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [39]:
import os
import numpy as np
import mne
import csv
import matplotlib.pyplot as plt

In [40]:
from sklearn.model_selection import train_test_split
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from keras.models import Sequential
from keras.layers import Dense, SpatialDropout1D
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix
from keras.layers import Dropout


# Set random seed for reproducibility
np.random.seed(42)

In [41]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [42]:
#path='/content/drive/MyDrive/Colab Notebooks/DL/'
path=r'/content/drive/MyDrive/Colab Notebooks/DL/ResampledData/'

length of the signal = observation time (minutes)* 60 (second/minutes) * sampling frequency (1/minutes)

end time = observation time (minutes) * 60 (second/minutes)




---



In [43]:
#observation_time=60 #in minutes
#sampling_frequency=200 #in Hertz or per second
number_of_data=500000
start_time=0
#nd_time=(observation_time*60)
#print(end_time)

We have valid data with total = 46

Create all zeros numpy arrays for FP and CZ signals

In [44]:
number_samples=46
signal_length=number_of_data
fp_cz_signals_o= np.zeros((number_samples, signal_length))

Read only FP1-A2 and CZ-A1:

   (20 subjects)

   (26 subjects) - Patient2 excluded, it does not have CZ signal



Apply bandpass filter:

  l_freq = 1  # low cutoff frequency in Hz

  h_freq = 45  # high cutoff frequency in Hz


Preprocessing: Scale the data to have zero mean and unit variance





In [45]:
file_name=[
                '/subject1_20_raw.fif','/subject2_20_raw.fif',
                '/subject3_20_raw.fif',
                '/subject4_20_raw.fif','/subject5_20_raw.fif',
                '/subject6_20_raw.fif','/subject7_20_raw.fif',
                '/subject8_20_raw.fif',
                '/subject9_20_raw.fif','/subject10_20_raw.fif',
                '/subject11_20_raw.fif','/subject12_20_raw.fif',
                '/subject13_20_raw.fif',
                '/subject14_20_raw.fif','/subject15_20_raw.fif',
                '/subject16_20_raw.fif','/subject17_20_raw.fif',
                '/subject18_20_raw.fif',
                '/subject19_20_raw.fif','/subject20_20_raw.fif',
                '/patient1_20_raw.fif',
                '/patient3_20_raw.fif',
                '/patient4_20_raw.fif','/patient5_20_raw.fif',
                '/patient6_20_raw.fif','/patient7_20_raw.fif',
                '/patient8_20_raw.fif',
                '/patient9_20_raw.fif','/patient10_20_raw.fif',
                '/patient11_20_raw.fif','/patient12_20_raw.fif',
                '/patient13_20_raw.fif',
                '/patient14_20_raw.fif','/patient15_20_raw.fif',
                '/patient16_20_raw.fif','/patient17_20_raw.fif',
                '/patient18_20_raw.fif',
                '/patient19_20_raw.fif','/patient20_20_raw.fif',
                '/patient21_20_raw.fif','/patient22_20_raw.fif',
                '/patient23_20_raw.fif',
                '/patient24_20_raw.fif','/patient25_20_raw.fif',
                '/patient26_20_raw.fif','/patient27_20_raw.fif',
                ]
channel_names = ['FP1-A2', 'CZ-A1']  # Channels are needed for this analysis
l_freq = 1  # low cutoff frequency in Hz
h_freq = 45  # high cutoff frequency in Hz
filter_length = 'auto'  # automatically calculate the filter length

for index in range(len(file_name)):
    #Read only FP1-A2 and CZ-A1
    raw = mne.io.read_raw_fif(path+file_name[index],preload=True,verbose=False)  # include=channel_names (only 'FP1-A2', 'CZ-A1')
    
    #Consider data length of the observation time from the beginning  
    #raw_filtered_cropped = raw.crop(tmin=start_time, tmax=end_time,include_tmax=False,verbose=False)
    raw_data=raw.get_data()
    channels = raw.ch_names
    #print(channels)
    b='FP1-A2'
    c='CZ-A1'
    Index_FP1_A2=channels.index(b)
    Index_CZ_A1=channels.index(c)
    #Read the data into fp_signals and cz-signals    
    fp_cz_signals_o[index] = raw_data[Index_FP1_A2]+raw_data[Index_CZ_A1]
#fp_cz_signals=(fp_cz_signals - np.mean(fp_cz_signals)) / np.std(fp_cz_signals)

#Create label for Sex: Woman and Man
#labels=np.array((1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,
#                 0,1,1,0,1,0,0,1,1,0,0,0,0,0,1,0,1,1,0,0,0,0,0,1,0,1))  #Sex: 1-woman, 0-man

#labels=np.array((1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
#                 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0))  # 1-Subject, 0-Patient

# Create label for Age:
labels=np.array((23,47,24,48,46,65,45,22,21,20,30,54,23,57,20,27,23,27,27,20,
                 72,52,42,40,74,52,31,53,61,73,47,38,48,53,45,57,52,53,19,44,38,60,50,27,71,69))  #Age
    

In [46]:
# Reshape input data to include a third dimension of size 1
fp_cz_signals=fp_cz_signals_o.reshape(fp_cz_signals_o.shape[0],fp_cz_signals_o.shape[1],1)

In [47]:
np.random.seed(42)

In [48]:
## Split the data into training and testing datasets for each electrodes (f:FP1-A2 and c:CZ-A1)
#Xfc_train, Xfc_test, yfc_train, yfc_test = train_test_split(fp_cz_signals, labels, test_size=0.2, random_state=42)


In [49]:
Xfc_train, Xfc_test, yfc_train, yfc_test = train_test_split(fp_cz_signals, labels, test_size=0.20, random_state=1)

#Xfc_train, Xfc_val, yfc_train, yfc_val = train_test_split(Xfc_train, yfc_train, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2


In [50]:
Xfc_train=(Xfc_train - np.mean(Xfc_train)) / np.std(Xfc_train)
#Xfc_val=(Xfc_val - np.mean(Xfc_train)) / np.std(Xfc_train)
Xfc_test=(Xfc_test - np.mean(Xfc_train)) / np.std(Xfc_train)

In [51]:
# Create a sequential model with a 1D convolutional layer
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(signal_length, 1)))
model.add(MaxPooling1D(pool_size=2))

model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())
model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=1, activation='linear'))

# Compile the model
model.compile(loss='mse', optimizer='adam', metrics=['mae'])

In [52]:
# Train the model for c:FP1-A2+CZ1-A1
#model.fit(X_train, y_train, epochs=10, batch_size=32)

#model.fit(Xfc_train, yfc_train, epochs=6, batch_size=32)
model.fit(Xfc_train, yfc_train, epochs=10, batch_size=64)

# Evaluate the model
#loss, accuracy = model.evaluate(X_test, y_test)
#loss, accuracy=model.evaluate(Xfc_val)

#print('Test Loss:', loss)
#print('Test Accuracy:', accuracy)

# Make predictions on new data
#y_pred = model.predict(X_test)
#y_pred = model.predict(Xfc_val)
#y_pred=np.ceil(y_pred)
#f1 = f1_score(yfc_val, y_pred)
#print("F1 score: {:.2f}%".format(f1*100))

mse, mae = model.evaluate(Xfc_test, yfc_test)




# Print the results
print("MSE: {:.2f}".format(mse))
print("MAE: {:.2f}".format(mae))
y_pred = model.predict(Xfc_test)
print(yfc_test)
print(y_pred)


Epoch 1/10
1/1 [==============================] - 31s 31s/step - loss: 2065.3354 - mae: 42.0677
Epoch 2/10
1/1 [==============================] - 29s 29s/step - loss: 818519.9375 - mae: 851.1582
Epoch 3/10
1/1 [==============================] - 21s 21s/step - loss: 38401.4805 - mae: 177.4855
Epoch 4/10
1/1 [==============================] - 19s 19s/step - loss: 5312.9326 - mae: 71.2144
Epoch 5/10
1/1 [==============================] - 19s 19s/step - loss: 2416.2561 - mae: 46.2158
Epoch 6/10
1/1 [==============================] - 19s 19s/step - loss: 2069.0513 - mae: 42.1124
Epoch 7/10
1/1 [==============================] - 20s 20s/step - loss: 2069.0774 - mae: 42.1127
Epoch 8/10
1/1 [==============================] - 19s 19s/step - loss: 2069.0984 - mae: 42.1129
Epoch 9/10
1/1 [==============================] - 19s 19s/step - loss: 2069.1147 - mae: 42.1131
Epoch 10/10
1/1 [==============================] - 1s 1s/step - loss: 2411.6160 - mae: 47.4023
MSE: 2411.62
MAE: 47.40
1/1 [=======

In [53]:
# Create a sequential model with a 1D convolutional layer
model = Sequential()

model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(signal_length, 1)))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())
model.add(Dense(units=32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=1, activation='linear'))



# Compile the model
model.compile(loss='mse', optimizer='adam', metrics=['mae'])

In [54]:
# Train the model for c:FP1-A2+CZ1-A1
#model.fit(X_train, y_train, epochs=10, batch_size=32)

#model.fit(Xfc_train, yfc_train, epochs=6, batch_size=32)
model.fit(Xfc_train, yfc_train, epochs=10, batch_size=64)

# Evaluate the model
#loss, accuracy = model.evaluate(X_test, y_test)
#loss, accuracy=model.evaluate(Xfc_val)

#print('Test Loss:', loss)
#print('Test Accuracy:', accuracy)

# Make predictions on new data
#y_pred = model.predict(X_test)
#y_pred = model.predict(Xfc_val)
#y_pred=np.ceil(y_pred)
#f1 = f1_score(yfc_val, y_pred)
#print("F1 score: {:.2f}%".format(f1*100))
# Train the model for c:FP1-A2+CZ1-A1
#model.fit(X_train, y_train, epochs=10, batch_size=32)

#model.fit(Xfc_train, yfc_train, epochs=6, batch_size=32)
model.fit(Xfc_train, yfc_train, epochs=10, batch_size=64)

# Evaluate the model
#loss, accuracy = model.evaluate(X_test, y_test)
#loss, accuracy=model.evaluate(Xfc_val)

#print('Test Loss:', loss)
#print('Test Accuracy:', accuracy)

# Make predictions on new data
#y_pred = model.predict(X_test)
#y_pred = model.predict(Xfc_val)
#y_pred=np.ceil(y_pred)
#f1 = f1_score(yfc_val, y_pred)
#print("F1 score: {:.2f}%".format(f1*100))

mse, mae = model.evaluate(Xfc_test, yfc_test)




# Print the results
print("MSE: {:.2f}".format(mse))
print("MAE: {:.2f}".format(mae))
y_pred = model.predict(Xfc_test)
print(yfc_test)
print(y_pred)

Epoch 1/10
1/1 [==============================] - 15s 15s/step - loss: 2080.6399 - mae: 42.2668
Epoch 2/10
1/1 [==============================] - 14s 14s/step - loss: 421731.0625 - mae: 608.3474
Epoch 3/10
1/1 [==============================] - 13s 13s/step - loss: 16275.3975 - mae: 112.9369
Epoch 4/10
1/1 [==============================] - 13s 13s/step - loss: 2068.9731 - mae: 42.1115
Epoch 5/10
1/1 [==============================] - 13s 13s/step - loss: 2069.0146 - mae: 42.1119
Epoch 6/10
1/1 [==============================] - 14s 14s/step - loss: 2069.0459 - mae: 42.1123
Epoch 7/10
1/1 [==============================] - 13s 13s/step - loss: 2069.0701 - mae: 42.1126
Epoch 8/10
1/1 [==============================] - 13s 13s/step - loss: 2069.0884 - mae: 42.1128
Epoch 9/10
1/1 [==============================] - 13s 13s/step - loss: 2069.1013 - mae: 42.1130
Epoch 10/10
1/1 [==============================] - 23s 23s/step - loss: 2069.1104 - mae: 42.1131
Epoch 1/10
1/1 [==================

In [55]:
# Create a sequential model with a 1D convolutional layer
model = Sequential()

model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(signal_length, 1)))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))


model.add(Flatten())
model.add(Dense(units=32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=1, activation='linear'))



# Compile the model
model.compile(loss='mse', optimizer='adam', metrics=['mae'])


In [56]:
# Train the model for c:FP1-A2+CZ1-A1
#model.fit(X_train, y_train, epochs=10, batch_size=32)

#model.fit(Xfc_train, yfc_train, epochs=6, batch_size=32)
model.fit(Xfc_train, yfc_train, epochs=10, batch_size=64)

# Evaluate the model
#loss, accuracy = model.evaluate(X_test, y_test)
#loss, accuracy=model.evaluate(Xfc_val)

#print('Test Loss:', loss)
#print('Test Accuracy:', accuracy)

# Make predictions on new data
#y_pred = model.predict(X_test)
#y_pred = model.predict(Xfc_val)
#y_pred=np.ceil(y_pred)
#f1 = f1_score(yfc_val, y_pred)
#print("F1 score: {:.2f}%".format(f1*100))

mse, mae = model.evaluate(Xfc_test, yfc_test)




# Print the results
print("MSE: {:.2f}".format(mse))
print("MAE: {:.2f}".format(mae))
y_pred = model.predict(Xfc_test)

print(yfc_test)
print(y_pred)

Epoch 1/10
1/1 [==============================] - 17s 17s/step - loss: 2085.9453 - mae: 42.3143
Epoch 2/10
1/1 [==============================] - 25s 25s/step - loss: 62221.5703 - mae: 222.6720
Epoch 3/10
1/1 [==============================] - 19s 19s/step - loss: 2943.9954 - mae: 42.9235
Epoch 4/10
1/1 [==============================] - 16s 16s/step - loss: 2068.9587 - mae: 42.1113
Epoch 5/10
1/1 [==============================] - 15s 15s/step - loss: 2068.9885 - mae: 42.1116
Epoch 6/10
1/1 [==============================] - 15s 15s/step - loss: 2069.0049 - mae: 42.1118
Epoch 7/10
1/1 [==============================] - 16s 16s/step - loss: 2069.0107 - mae: 42.1119
Epoch 8/10
1/1 [==============================] - 15s 15s/step - loss: 2069.0088 - mae: 42.1119
Epoch 9/10
1/1 [==============================] - 15s 15s/step - loss: 2069.0000 - mae: 42.1118
Epoch 10/10
1/1 [==============================] - 1s 1s/step - loss: 2411.4238 - mae: 47.4002
MSE: 2411.42
MAE: 47.40
1/1 [==========

In [57]:
# Create a sequential model with a 1D convolutional layer
model = Sequential()

model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(signal_length, 1)))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())
model.add(Dense(units=64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=1, activation='linear'))



# Compile the model
model.compile(loss='mse', optimizer='adam', metrics=['mae'])



In [58]:
# Train the model for c:FP1-A2+CZ1-A1
#model.fit(X_train, y_train, epochs=10, batch_size=32)

#model.fit(Xfc_train, yfc_train, epochs=6, batch_size=32)
model.fit(Xfc_train, yfc_train, epochs=10, batch_size=64)

# Evaluate the model
#loss, accuracy = model.evaluate(X_test, y_test)
#loss, accuracy=model.evaluate(Xfc_val)

#print('Test Loss:', loss)
#print('Test Accuracy:', accuracy)

# Make predictions on new data
#y_pred = model.predict(X_test)
#y_pred = model.predict(Xfc_val)
#y_pred=np.ceil(y_pred)
#f1 = f1_score(yfc_val, y_pred)
#print("F1 score: {:.2f}%".format(f1*100))

mse, mae = model.evaluate(Xfc_test, yfc_test)




# Print the results
print("MSE: {:.2f}".format(mse))
print("MAE: {:.2f}".format(mae))
y_pred = model.predict(Xfc_test)

print(yfc_test)
print(y_pred)


Epoch 1/10
1/1 [==============================] - 23s 23s/step - loss: 2069.7744 - mae: 42.1306
Epoch 2/10
1/1 [==============================] - 20s 20s/step - loss: 31742.7324 - mae: 159.5004
Epoch 3/10
1/1 [==============================] - 20s 20s/step - loss: 1016.1600 - mae: 24.7870
Epoch 4/10
1/1 [==============================] - 21s 21s/step - loss: 2068.9453 - mae: 42.1111
Epoch 5/10
1/1 [==============================] - 20s 20s/step - loss: 2068.9639 - mae: 42.1113
Epoch 6/10
1/1 [==============================] - 21s 21s/step - loss: 2068.9683 - mae: 42.1114
Epoch 7/10
1/1 [==============================] - 20s 20s/step - loss: 2068.9619 - mae: 42.1113
Epoch 8/10
1/1 [==============================] - 20s 20s/step - loss: 2068.9463 - mae: 42.1111
Epoch 9/10
1/1 [==============================] - 20s 20s/step - loss: 2068.9238 - mae: 42.1109
Epoch 10/10
1/1 [==============================] - 1s 1s/step - loss: 2411.3074 - mae: 47.3990
MSE: 2411.31
MAE: 47.40
1/1 [==========

In [59]:
# Create a sequential model with a 1D convolutional layer
model = Sequential()

model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(signal_length, 1)))
model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())
model.add(Dense(units=32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=1, activation='linear'))



# Compile the model
model.compile(loss='mse', optimizer='adam', metrics=['mae'])

In [60]:
# Train the model for c:FP1-A2+CZ1-A1
#model.fit(X_train, y_train, epochs=10, batch_size=32)

#model.fit(Xfc_train, yfc_train, epochs=6, batch_size=32)
model.fit(Xfc_train, yfc_train, epochs=10, batch_size=64)

# Evaluate the model
#loss, accuracy = model.evaluate(X_test, y_test)
#loss, accuracy=model.evaluate(Xfc_val)

#print('Test Loss:', loss)
#print('Test Accuracy:', accuracy)

# Make predictions on new data
#y_pred = model.predict(X_test)
mse, mae = model.evaluate(Xfc_test, yfc_test)




# Print the results
print("MSE: {:.2f}".format(mse))
print("MAE: {:.2f}".format(mae))
y_pred = model.predict(Xfc_test)
print(yfc_test)
print(y_pred)


Epoch 1/10
1/1 [==============================] - 11s 11s/step - loss: 2060.5022 - mae: 42.0429
Epoch 2/10
1/1 [==============================] - 8s 8s/step - loss: 7264391.0000 - mae: 2543.5215
Epoch 3/10
1/1 [==============================] - 9s 9s/step - loss: 433566.8438 - mae: 616.7115
Epoch 4/10
1/1 [==============================] - 9s 9s/step - loss: 2068.9810 - mae: 42.1115
Epoch 5/10
1/1 [==============================] - 9s 9s/step - loss: 2069.0286 - mae: 42.1121
Epoch 6/10
1/1 [==============================] - 9s 9s/step - loss: 2069.0681 - mae: 42.1126
Epoch 7/10
1/1 [==============================] - 8s 8s/step - loss: 2069.1013 - mae: 42.1130
Epoch 8/10
1/1 [==============================] - 9s 9s/step - loss: 2069.1296 - mae: 42.1133
Epoch 9/10
1/1 [==============================] - 8s 8s/step - loss: 2069.1541 - mae: 42.1136
Epoch 10/10
1/1 [==============================] - 1s 735ms/step - loss: 2411.6799 - mae: 47.4030
MSE: 2411.68
MAE: 47.40
1/1 [=================

In [61]:
# Create a sequential model with a 1D convolutional layer - 4
model = Sequential()

model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(signal_length, 1)))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())
model.add(Dense(units=64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=1, activation='linear'))



# Compile the model
model.compile(loss='mse', optimizer='adam', metrics=['mae'])


In [62]:
# Train the model for c:FP1-A2+CZ1-A1
#model.fit(X_train, y_train, epochs=10, batch_size=32)

#model.fit(Xfc_train, yfc_train, epochs=6, batch_size=32)
model.fit(Xfc_train, yfc_train, epochs=10, batch_size=64)

# Evaluate the model
#loss, accuracy = model.evaluate(X_test, y_test)
#loss, accuracy=model.evaluate(Xfc_val)

#print('Test Loss:', loss)
#print('Test Accuracy:', accuracy)

# Make predictions on new data
#y_pred = model.predict(X_test)
mse, mae = model.evaluate(Xfc_test, yfc_test)




# Print the results
print("MSE: {:.2f}".format(mse))
print("MAE: {:.2f}".format(mae))
y_pred = model.predict(Xfc_test)
print(yfc_test)
print(y_pred)


Epoch 1/10
1/1 [==============================] - 23s 23s/step - loss: 2057.9756 - mae: 41.9804
Epoch 2/10
1/1 [==============================] - 21s 21s/step - loss: 14989.2227 - mae: 107.9248
Epoch 3/10
1/1 [==============================] - 20s 20s/step - loss: 867.7076 - mae: 23.3223
Epoch 4/10
1/1 [==============================] - 21s 21s/step - loss: 1966.8939 - mae: 40.3021
Epoch 5/10
1/1 [==============================] - 20s 20s/step - loss: 1861.5890 - mae: 39.4551
Epoch 6/10
1/1 [==============================] - 33s 33s/step - loss: 1610.3914 - mae: 35.3963
Epoch 7/10
1/1 [==============================] - 21s 21s/step - loss: 1375.7986 - mae: 32.1112
Epoch 8/10
1/1 [==============================] - 21s 21s/step - loss: 1316.5028 - mae: 31.1149
Epoch 9/10
1/1 [==============================] - 20s 20s/step - loss: 1194.8251 - mae: 28.7820
Epoch 10/10
1/1 [==============================] - 2s 2s/step - loss: 599605.6875 - mae: 718.3432
MSE: 599605.69
MAE: 718.34
1/1 [=====

In [63]:
# Create a sequential model with a 1D convolutional layer - 5
model = Sequential()

model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(signal_length, 1)))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())
model.add(Dense(units=64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=1, activation='linear'))



# Compile the model
model.compile(loss='mse', optimizer='adam', metrics=['mae'])


In [64]:
# Train the model for c:FP1-A2+CZ1-A1
#model.fit(X_train, y_train, epochs=10, batch_size=32)

#model.fit(Xfc_train, yfc_train, epochs=6, batch_size=32)
model.fit(Xfc_train, yfc_train, epochs=10, batch_size=64)

# Evaluate the model
#loss, accuracy = model.evaluate(X_test, y_test)
#loss, accuracy=model.evaluate(Xfc_val)

#print('Test Loss:', loss)
#print('Test Accuracy:', accuracy)

# Make predictions on new data
#y_pred = model.predict(X_test)
mse, mae = model.evaluate(Xfc_test, yfc_test)




# Print the results
print("MSE: {:.2f}".format(mse))
print("MAE: {:.2f}".format(mae))
y_pred = model.predict(Xfc_test)
print(yfc_test)
print(y_pred)


Epoch 1/10
1/1 [==============================] - 21s 21s/step - loss: 2061.1824 - mae: 42.0073
Epoch 2/10
1/1 [==============================] - 21s 21s/step - loss: 874.3147 - mae: 23.8629
Epoch 3/10
1/1 [==============================] - 23s 23s/step - loss: 643.1396 - mae: 20.7886
Epoch 4/10
1/1 [==============================] - 20s 20s/step - loss: 630.5029 - mae: 20.2626
Epoch 5/10
1/1 [==============================] - 19s 19s/step - loss: 486.8583 - mae: 17.5005
Epoch 6/10
1/1 [==============================] - 19s 19s/step - loss: 501.9130 - mae: 19.3799
Epoch 7/10
1/1 [==============================] - 19s 19s/step - loss: 397.6705 - mae: 16.5687
Epoch 8/10
1/1 [==============================] - 20s 20s/step - loss: 401.3901 - mae: 15.9327
Epoch 9/10
1/1 [==============================] - 19s 19s/step - loss: 345.2274 - mae: 15.4232
Epoch 10/10
1/1 [==============================] - 2s 2s/step - loss: 1495442.1250 - mae: 1164.7490
MSE: 1495442.12
MAE: 1164.75
1/1 [==========

In [65]:
# Create a sequential model with a 1D convolutional layer - 6
model = Sequential()

model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(signal_length, 1)))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())
model.add(Dense(units=64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=1, activation='linear'))



# Compile the model
model.compile(loss='mse', optimizer='adam', metrics=['mae'])

In [66]:
# Train the model for c:FP1-A2+CZ1-A1
#model.fit(X_train, y_train, epochs=10, batch_size=32)

#model.fit(Xfc_train, yfc_train, epochs=6, batch_size=32)
model.fit(Xfc_train, yfc_train, epochs=10, batch_size=64)

# Evaluate the model
#loss, accuracy = model.evaluate(X_test, y_test)
#loss, accuracy=model.evaluate(Xfc_val)

#print('Test Loss:', loss)
#print('Test Accuracy:', accuracy)

# Make predictions on new data
#y_pred = model.predict(X_test)
mse, mae = model.evaluate(Xfc_test, yfc_test)




# Print the results
print("MSE: {:.2f}".format(mse))
print("MAE: {:.2f}".format(mae))
y_pred = model.predict(Xfc_test)
print(yfc_test)
print(y_pred)

Epoch 1/10
1/1 [==============================] - 21s 21s/step - loss: 2071.6494 - mae: 42.1492
Epoch 2/10
1/1 [==============================] - 19s 19s/step - loss: 1568.1556 - mae: 35.0908
Epoch 3/10
1/1 [==============================] - 20s 20s/step - loss: 880.1581 - mae: 24.4350
Epoch 4/10
1/1 [==============================] - 19s 19s/step - loss: 714.6599 - mae: 22.2450
Epoch 5/10
1/1 [==============================] - 20s 20s/step - loss: 1135.5582 - mae: 25.9599
Epoch 6/10
1/1 [==============================] - 20s 20s/step - loss: 617.8145 - mae: 20.5876
Epoch 7/10
1/1 [==============================] - 19s 19s/step - loss: 594.5112 - mae: 20.4484
Epoch 8/10
1/1 [==============================] - 19s 19s/step - loss: 601.7133 - mae: 19.8648
Epoch 9/10
1/1 [==============================] - 19s 19s/step - loss: 673.3854 - mae: 20.7479
Epoch 10/10
1/1 [==============================] - 2s 2s/step - loss: 675591.3750 - mae: 785.1162
MSE: 675591.38
MAE: 785.12
1/1 [============

In [67]:
# Create a sequential model with a 1D convolutional layer - 7
model = Sequential()

model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(signal_length, 1)))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())
model.add(Dense(units=64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=1, activation='linear'))



# Compile the model
model.compile(loss='mse', optimizer='adam', metrics=['mae'])

In [68]:
# Train the model for c:FP1-A2+CZ1-A1
#model.fit(X_train, y_train, epochs=10, batch_size=32)

#model.fit(Xfc_train, yfc_train, epochs=6, batch_size=32)
model.fit(Xfc_train, yfc_train, epochs=10, batch_size=64)

# Evaluate the model
#loss, accuracy = model.evaluate(X_test, y_test)
#loss, accuracy=model.evaluate(Xfc_val)

#print('Test Loss:', loss)
#print('Test Accuracy:', accuracy)

# Make predictions on new data
#y_pred = model.predict(X_test)
mse, mae = model.evaluate(Xfc_test, yfc_test)




# Print the results
print("MSE: {:.2f}".format(mse))
print("MAE: {:.2f}".format(mae))
y_pred = model.predict(Xfc_test)
print(yfc_test)
print(y_pred)


Epoch 1/10
1/1 [==============================] - 21s 21s/step - loss: 2067.9365 - mae: 42.0969
Epoch 2/10
1/1 [==============================] - 24s 24s/step - loss: 1270.4744 - mae: 30.2756
Epoch 3/10
1/1 [==============================] - 25s 25s/step - loss: 616.9880 - mae: 20.7514
Epoch 4/10
1/1 [==============================] - 26s 26s/step - loss: 2296.1399 - mae: 37.0504
Epoch 5/10
1/1 [==============================] - 19s 19s/step - loss: 698.4276 - mae: 21.2101
Epoch 6/10
1/1 [==============================] - 18s 18s/step - loss: 612.8229 - mae: 20.1683
Epoch 7/10
1/1 [==============================] - 19s 19s/step - loss: 824.2558 - mae: 23.1348
Epoch 8/10
1/1 [==============================] - 19s 19s/step - loss: 947.9477 - mae: 25.5048
Epoch 9/10
1/1 [==============================] - 19s 19s/step - loss: 1011.7641 - mae: 26.3226
Epoch 10/10
1/1 [==============================] - 1s 1s/step - loss: 256082.2031 - mae: 472.8883
MSE: 256082.20
MAE: 472.89
1/1 [===========

In [69]:
# Create a sequential model with a 1D convolutional layer - 8
model = Sequential()

model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(signal_length, 1)))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())
model.add(Dense(units=64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=1, activation='linear'))



# Compile the model
model.compile(loss='mse', optimizer='adam', metrics=['mae'])

In [70]:
# Train the model for c:FP1-A2+CZ1-A1
#model.fit(X_train, y_train, epochs=10, batch_size=32)

#model.fit(Xfc_train, yfc_train, epochs=6, batch_size=32)
model.fit(Xfc_train, yfc_train, epochs=10, batch_size=64)

# Evaluate the model
#loss, accuracy = model.evaluate(X_test, y_test)
#loss, accuracy=model.evaluate(Xfc_val)

#print('Test Loss:', loss)
#print('Test Accuracy:', accuracy)

# Make predictions on new data
#y_pred = model.predict(X_test)
mse, mae = model.evaluate(Xfc_test, yfc_test)




# Print the results
print("MSE: {:.2f}".format(mse))
print("MAE: {:.2f}".format(mae))
y_pred = model.predict(Xfc_test)
print(yfc_test)
print(y_pred)

Epoch 1/10
1/1 [==============================] - 23s 23s/step - loss: 2076.7041 - mae: 42.2141
Epoch 2/10
1/1 [==============================] - 19s 19s/step - loss: 1877.6448 - mae: 39.5966
Epoch 3/10
1/1 [==============================] - 18s 18s/step - loss: 1535.4733 - mae: 34.5862
Epoch 4/10
1/1 [==============================] - 18s 18s/step - loss: 1062.4609 - mae: 27.2095
Epoch 5/10
1/1 [==============================] - 19s 19s/step - loss: 602.0193 - mae: 20.3218
Epoch 6/10
1/1 [==============================] - 19s 19s/step - loss: 1208.6876 - mae: 26.7310
Epoch 7/10
1/1 [==============================] - 18s 18s/step - loss: 891.1897 - mae: 22.5136
Epoch 8/10
1/1 [==============================] - 18s 18s/step - loss: 600.8130 - mae: 19.9969
Epoch 9/10
1/1 [==============================] - 19s 19s/step - loss: 625.2381 - mae: 20.3350
Epoch 10/10
1/1 [==============================] - 2s 2s/step - loss: 363119.9062 - mae: 562.5164
MSE: 363119.91
MAE: 562.52
1/1 [==========

In [71]:
# Create a sequential model with a 1D convolutional layer - 9
model = Sequential()

model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(signal_length, 1)))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())
model.add(Dense(units=64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=1, activation='linear'))



# Compile the model
model.compile(loss='mse', optimizer='adam', metrics=['mae'])

In [72]:
# Train the model for c:FP1-A2+CZ1-A1
#model.fit(X_train, y_train, epochs=10, batch_size=32)

#model.fit(Xfc_train, yfc_train, epochs=6, batch_size=32)
model.fit(Xfc_train, yfc_train, epochs=10, batch_size=64)

# Evaluate the model
#loss, accuracy = model.evaluate(X_test, y_test)
#loss, accuracy=model.evaluate(Xfc_val)

#print('Test Loss:', loss)
#print('Test Accuracy:', accuracy)

# Make predictions on new data
#y_pred = model.predict(X_test)
mse, mae = model.evaluate(Xfc_test, yfc_test)




# Print the results
print("MSE: {:.2f}".format(mse))
print("MAE: {:.2f}".format(mae))
y_pred = model.predict(Xfc_test)
print(yfc_test)
print(y_pred)

Epoch 1/10
1/1 [==============================] - 23s 23s/step - loss: 2069.2236 - mae: 42.1129
Epoch 2/10
1/1 [==============================] - 19s 19s/step - loss: 1996.1901 - mae: 41.1775
Epoch 3/10
1/1 [==============================] - 19s 19s/step - loss: 1861.3376 - mae: 39.3993
Epoch 4/10
1/1 [==============================] - 19s 19s/step - loss: 1622.0474 - mae: 35.8953
Epoch 5/10
1/1 [==============================] - 20s 20s/step - loss: 1257.6866 - mae: 29.9481
Epoch 6/10
1/1 [==============================] - 19s 19s/step - loss: 822.4502 - mae: 22.9579
Epoch 7/10
1/1 [==============================] - 19s 19s/step - loss: 686.7963 - mae: 22.1419
Epoch 8/10
1/1 [==============================] - 20s 20s/step - loss: 1113.0288 - mae: 23.6736
Epoch 9/10
1/1 [==============================] - 19s 19s/step - loss: 885.1343 - mae: 23.6092
Epoch 10/10
1/1 [==============================] - 2s 2s/step - loss: 1147479.0000 - mae: 1002.3749
MSE: 1147479.00
MAE: 1002.37
1/1 [=====

In [73]:
# Create a sequential model with a 1D convolutional layer - 10
model = Sequential()

model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(signal_length, 1)))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))


model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())
model.add(Dense(units=64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=1, activation='linear'))



# Compile the model
model.compile(loss='mse', optimizer='adam', metrics=['mae'])

In [74]:
# Train the model for c:FP1-A2+CZ1-A1
#model.fit(X_train, y_train, epochs=10, batch_size=32)

#model.fit(Xfc_train, yfc_train, epochs=6, batch_size=32)
model.fit(Xfc_train, yfc_train, epochs=10, batch_size=64)

# Evaluate the model
#loss, accuracy = model.evaluate(X_test, y_test)
#loss, accuracy=model.evaluate(Xfc_val)

#print('Test Loss:', loss)
#print('Test Accuracy:', accuracy)

# Make predictions on new data
#y_pred = model.predict(X_test)
mse, mae = model.evaluate(Xfc_test, yfc_test)




# Print the results
print("MSE: {:.2f}".format(mse))
print("MAE: {:.2f}".format(mae))
y_pred = model.predict(Xfc_test)
print(y_pred)
print(yfc_test)


Epoch 1/10
1/1 [==============================] - 35s 35s/step - loss: 2071.0552 - mae: 42.1348
Epoch 2/10
1/1 [==============================] - 19s 19s/step - loss: 2039.7316 - mae: 41.7355
Epoch 3/10
1/1 [==============================] - 18s 18s/step - loss: 1975.4213 - mae: 40.9064
Epoch 4/10
1/1 [==============================] - 20s 20s/step - loss: 1827.0435 - mae: 38.8988
Epoch 5/10
1/1 [==============================] - 19s 19s/step - loss: 1596.7515 - mae: 35.5809
Epoch 6/10
1/1 [==============================] - 18s 18s/step - loss: 1191.7478 - mae: 29.1291
Epoch 7/10
1/1 [==============================] - 18s 18s/step - loss: 773.5837 - mae: 22.2613
Epoch 8/10
1/1 [==============================] - 18s 18s/step - loss: 683.7000 - mae: 20.4925
Epoch 9/10
1/1 [==============================] - 19s 19s/step - loss: 1214.1401 - mae: 26.2672
Epoch 10/10
1/1 [==============================] - 2s 2s/step - loss: 1608228.1250 - mae: 1162.7804
MSE: 1608228.12
MAE: 1162.78
1/1 [====

In [75]:
# Create a sequential model with a 1D convolutional layer - 11
model = Sequential()

model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(signal_length, 1)))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))


model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())
model.add(Dense(units=64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=1, activation='linear'))



# Compile the model
model.compile(loss='mse', optimizer='adam', metrics=['mae'])

In [76]:
# Train the model for c:FP1-A2+CZ1-A1
#model.fit(X_train, y_train, epochs=10, batch_size=32)

#model.fit(Xfc_train, yfc_train, epochs=6, batch_size=32)
model.fit(Xfc_train, yfc_train, epochs=10, batch_size=64)

# Evaluate the model
#loss, accuracy = model.evaluate(X_test, y_test)
#loss, accuracy=model.evaluate(Xfc_val)

#print('Test Loss:', loss)
#print('Test Accuracy:', accuracy)

# Make predictions on new data
#y_pred = model.predict(X_test)
mse, mae = model.evaluate(Xfc_test, yfc_test)




# Print the results
print("MSE: {:.2f}".format(mse))
print("MAE: {:.2f}".format(mae))
y_pred = model.predict(Xfc_test)
print(y_pred)
print(yfc_test)


Epoch 1/10
1/1 [==============================] - 22s 22s/step - loss: 2066.7048 - mae: 42.0812
Epoch 2/10
1/1 [==============================] - 19s 19s/step - loss: 2044.5229 - mae: 41.8011
Epoch 3/10
1/1 [==============================] - 18s 18s/step - loss: 1990.0265 - mae: 41.1084
Epoch 4/10
1/1 [==============================] - 18s 18s/step - loss: 1881.7552 - mae: 39.7207
Epoch 5/10
1/1 [==============================] - 18s 18s/step - loss: 1679.7311 - mae: 36.8732
Epoch 6/10
1/1 [==============================] - 19s 19s/step - loss: 1346.0610 - mae: 31.7586
Epoch 7/10
1/1 [==============================] - 22s 22s/step - loss: 881.1413 - mae: 24.0470
Epoch 8/10
1/1 [==============================] - 19s 19s/step - loss: 574.1503 - mae: 20.4600
Epoch 9/10
1/1 [==============================] - 18s 18s/step - loss: 1164.9741 - mae: 25.9424
Epoch 10/10
1/1 [==============================] - 2s 2s/step - loss: 1784555.6250 - mae: 1241.5891
MSE: 1784555.62
MAE: 1241.59
1/1 [====

In [77]:
# Create a sequential model with a 1D convolutional layer - 5 - 2
model = Sequential()

model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(signal_length, 1)))
model.add(MaxPooling1D(pool_size=2))

model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())
model.add(Dense(units=64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=1, activation='linear'))



# Compile the model
model.compile(loss='mse', optimizer='adam', metrics=['mae'])


In [78]:
# Train the model for c:FP1-A2+CZ1-A1
#model.fit(X_train, y_train, epochs=10, batch_size=32)

#model.fit(Xfc_train, yfc_train, epochs=6, batch_size=32)
model.fit(Xfc_train, yfc_train, epochs=10, batch_size=64)

# Evaluate the model
#loss, accuracy = model.evaluate(X_test, y_test)
#loss, accuracy=model.evaluate(Xfc_val)

#print('Test Loss:', loss)
#print('Test Accuracy:', accuracy)

# Make predictions on new data
#y_pred = model.predict(X_test)
mse, mae = model.evaluate(Xfc_test, yfc_test)




# Print the results
print("MSE: {:.2f}".format(mse))
print("MAE: {:.2f}".format(mae))
y_pred = model.predict(Xfc_test)
print(y_pred)
print(yfc_test)

Epoch 1/10
1/1 [==============================] - 30s 30s/step - loss: 2076.9028 - mae: 42.2237
Epoch 2/10
1/1 [==============================] - 39s 39s/step - loss: 705.0753 - mae: 21.5338
Epoch 3/10
1/1 [==============================] - 27s 27s/step - loss: 986.5825 - mae: 23.7520
Epoch 4/10
1/1 [==============================] - 26s 26s/step - loss: 604.2870 - mae: 20.4473
Epoch 5/10
1/1 [==============================] - 25s 25s/step - loss: 635.4583 - mae: 20.3228
Epoch 6/10
1/1 [==============================] - 26s 26s/step - loss: 624.7996 - mae: 19.4378
Epoch 7/10
1/1 [==============================] - 26s 26s/step - loss: 486.0081 - mae: 17.1684
Epoch 8/10
1/1 [==============================] - 25s 25s/step - loss: 480.3639 - mae: 18.6211
Epoch 9/10
1/1 [==============================] - 26s 26s/step - loss: 425.4750 - mae: 16.3652
Epoch 10/10
1/1 [==============================] - 2s 2s/step - loss: 860967.6875 - mae: 894.5801
MSE: 860967.69
MAE: 894.58
1/1 [==============